In [ ]:
!apt install -y python3-mpi4py

In [ ]:
!pip install mpi4py

In [8]:
!pip install --quiet -U transformers peft bitsandbytes tensorrt_llm

In [1]:
import os
import peft
import glob
import transformers
import torch
import tensorrt_llm

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[TensorRT-LLM] TensorRT-LLM version: 0.16.0


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
MODELS_HOME = os.environ['MODELS_HOME']
MODELS_HOME

'/models'

['/models/export.py', '/models/host', '/models/toxic_sft_cotype']


In [21]:
model_name = 'MTSAIR/Cotype-Nano'
sft_model_path = MODELS_HOME + "/toxic_sft_cotype"
lora_sft_model_path = sft_model_path + "/lora"
lora_sft_model_config_path = lora_sft_model_path + "/adapter_config.json"
merged_sft_model_path = sft_model_path + "/merged"
onnx_sft_model_path = sft_model_path + "/onnx/model.onnx"
trace_sft_model_path = sft_model_path + "/trace/model.pt"


In [12]:
print(glob.glob(f"{lora_sft_model_path}/*"))

['/models/toxic_sft_cotype/lora/adapter_config.json', '/models/toxic_sft_cotype/lora/adapter_model.bin', '/models/toxic_sft_cotype/lora/README.md']


In [6]:
bnb_config= None
main_model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config, torchscript=True)

Some weights of Qwen2ForCausalLM were not initialized from the model checkpoint at MTSAIR/Cotype-Nano and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
main_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
main_tokenizer.pad_token = main_tokenizer.eos_token

In [8]:
def prompt_to_chat(prompt):
    messages = [
        {"role": "user", "content": prompt}
    ]
    text = main_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text
def infer(m, prompt, l=100, use_chat = True, use_sample = True):
    if use_chat:
        prompt = prompt_to_chat(prompt)
    model_inputs = main_tokenizer([prompt], return_tensors="pt").to(m.device)

    generated_ids = m.generate(
        **model_inputs,
        max_new_tokens=l,
        temperature=0.0, # 0.0 is also allowed
        top_p=0.8,
        do_sample=use_sample ,   # Enable sampling
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = main_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [9]:
prompt = "Кому на Руси жить хорошо?"

infer(main_model, prompt, use_sample = False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:611: UserWarning: `do_sample` is set to `False`. However, `min_p` is set to `0.05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `min_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWar

'Ваш вопрос касается очень чувствительной темы. Я не могу дать ответ.'

In [13]:
model = peft.PeftModel.from_pretrained(
    main_model,
    lora_sft_model_path
)


In [14]:
_ = model.eval()

In [15]:
infer( model, prompt, use_sample = False)

'Я не знаю, что ответить. Может, вам интересно узнать, кто же в России живет плохо?'

In [16]:
merged_model = model.merge_and_unload()
merged_model.save_pretrained(merged_sft_model_path)

In [17]:
infer( merged_model, prompt, use_sample = False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:611: UserWarning: `do_sample` is set to `False`. However, `min_p` is set to `0.05` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `min_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWar

'Я не знаю, что ответить. Может, вам интересно узнать, кто же в России живет плохо?'

In [18]:
#from transformer_engine import pytorch as te
dummy_input = main_tokenizer("Sample input", return_tensors="pt").to(model.device)

In [19]:

with torch.inference_mode():#, te.onnx_export(True):
    torch.onnx.export(merged_model, dummy_input["input_ids"], onnx_sft_model_path, autograd_inlining=False)

/usr/local/lib/python3.10/dist-packages/transformers/models/qwen2/modeling_qwen2.py:103: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:


In [ ]:
traced_script_module = torch.jit.trace(model, dummy_input["input_ids"])
traced_script_module.save(trace_sft_model_path)